In [1]:
%run Latex_macros.ipynb

<IPython.core.display.Latex object>

# RNN in action: Understanding sequences

We will study a toy example that is typical of many tasks involving sequences
- Given a prefix of a sequence
- Predict the next element

For example
- Predict the next word in a sentence
- Predict the next price in a timeseries of prices

Being able to predict the next element may be key to understanding the "logic" underlying a sequence
- You have to understand context and domain
- You have to understand how earlier elements influence latter elements

## Predict the next:  Data preparation

It is our belief that Machine Learning is a *process* and not just a collection of models.

We have recently been emphasizing the models but let's review the process.



<table>
    <tr>
        <th><center>Recipe for Machine Learning</center></th>
    </tr>
    <tr>
        <td><img src="images/ML_process.jpg"</td>
    </tr>
</table>


It is usually the case that Sequence data involves substantial Data Preparation.

Suppose our task is to predict the next word in a sentence.

We are given (or must obtain) a collection of sentences (e.g., one or more documents) as our raw data.

But a sentence is not the format required for the training set of the "Predict the next word" task.

Data preparation is usually a substantial prerequisite for solving tasks involving sequences.

To be precises, the "Predict the next word" task involves
- Training a many to one RNN with examples created from a sequence.
- The elements of a single example are the prefix of a sentence
- The target of the example is the next word in the sentence


Let 
$$[ \;\mathbf{s}_\tp | 1 \le \tt \le T \; ]$$
be the sequence of words in sentence \mathbf{s}.

We will prepare $(T-1)$ examples from this single sentence.


$\langle \X, \y \rangle = $

$
\begin{array} \\
  i  & \x^\ip  & \y^\ip \\
  \hline \\
  1 & \mathbf{s}_{(1) }  & \mathbf{s}_{(2)} \\
  2 & \mathbf{s}_{(1), (2) }  & \mathbf{s}_{(3)} \\
  \vdots \\
  i & \mathbf{s}_{(1), \ldots, (i) }  & \mathbf{s}_{(i+1)} \\
  \vdots \\
  (T-1) & \mathbf{s}_{(1), \ldots, (T-1) }  & \mathbf{s}_{(T)} \\
  \end{array}
$

For example

$\mathbf{s} = $
"I am taking a class in Machine Learning"


$
\begin{array}\\
i & \x^\ip  & \y^\ip \\
1 & [\;  \text{I} \; ] & \text{am} \\
2 & [\; \text{I, am} \; ] & \text{taking} \\
3 & [\; \text{I, am, taking} \; ] & \text{a} \\
\end{array}
$

## Predict the next: data shape

We had warned earlier about the explosion of th enumber of dimensions of our data.
Now is a good time to take stock
- $\X$, the training set, is a matrix with $m$ rows
- Each row is an example $\x^\ip$
- Each example is a sequence $[ \; \x^\ip_\tp \, | \, 1 \le \tt \le || \x^\ip || \; ]$
- Each element $\x^\ip_\tp$ of the sequence encodes a word
- A word is encoded as a One Hot Encoded binary vector of length $|| V||$ where $V$ is the set of words in the vocabulary

Target $\y^\ip$ is also a word (so is  vector of length $|| V||$).
- Many to one: target is *not* a sequence

## Predict the next: training

Just like training any other type of layer, but more expensive
- Each example involves multiple time steps: forward pass time consuming
- The derivatives (needed for Gradient Descent) are more complex; backward pass complex and time consuming


# RNN as a generative model (fun with RNN's)

The "Predict the next" word task is interesting on its own
- But a slight twist will make it extremely interesting

Suppose
- We train the RNN on a large number of sentences of the same type (e.g., same author)
- Create a few words to create the prefix of a sentence
- Ask the RNN to predict the next word
- Append this word to the prefix
- Repeat !

Voila: the RNN can *generate* a story in the same style as the training sentences.

Using Machine Learning to *create* data is called *generative*.

Using Machine Learning to classify/predict (as we've been doing thus far) is called *discriminative*.

## Architecture

How do we construct a model to solve this task ?

We construct a two part model in what is known as an *Encoder/Decoder* architecture

The *Encoder* is a many to one RNN
- Takes the variable length "seed" sequence
- Outputs a fixed length representation of the seed
    - This is one of the strengths of an RNN
    

The *Decoder* is a one to many RNN
- Takes the fixed length representation of the seed produced by the Encoder
    - Used to initialized the Decoder's latent state $\h_{(0)}$
- Outputs a variable length sequence

The only thing unusual about the Decoder is how its input sequence is constructed
- All the elements of its input $\x$ are *generated* by the Decoder
- The first element of $\x$ is set to a special "start of output" symbol
$$\x_{(1)} = \langle \text{START} \rangle$$
- $\x_\tp$ is extending *dynamically*, using the previous prediction $\hat{\y}_{(\tt-1)}$
$$
\x^\ip_\tp = \y_{(t-1)}
$$
- The Decoder stops when it generates a special "end of output" symbol $\langle \text{END} \rangle$

<table>
    <tr>
        <th><center>Sequence to Sequence: inference</center></th>
    </tr>
    <tr>
        <td><img src="images/RNN_seq2seq_inference.jpg" width=1000></td>
    </tr>
</table>

That is
- The output is fed as the next input
- Thus extending the sequence
- And making sure that subsequent elements are influenced by all previously *generated* elements
- Continuing until the special $\langle \text{END} \rangle$ symbol is generated

## Training: Teacher forcing

Some issues arise in using an RNN in the generative manner.

The first issue:
- Is the prediction a single word or a probability distribution over the vocabulary $||V||$
- If it's a single word: the output is deterministic
    - Problematic once one word is wrong: the error propagates forward

The output of the multinomial classifier is a vector of length $||V||$
- With values in the range $[0,1]$ that can be interpretted as probabilities
- Rather than choosing the single word with highest probability
- We *sample* one word at random, according to the probability distribution

This makes the output non-deterministic: running the model twice with the same "seed" may give different stories.

A second issue
- If a wrong word is chosen at step $\tt$: it affects the generation of all words at step $\tt \gt \tt'$ 
- This is particularly problematic at *training* time: makes learning difficult

The solution for training an RNN for this task is a method known as *teacher forcing*
- Rather than extending the seed example $\x^\ip = [ \; \x^\ip_\tp \, | \, 1 \le \tt \le t' \; ]$
    - With $\hat{\y}_\tp$, the *predicted* $\tt^{th}$ word for $(\tt > \tt')$
    - Which is what would happen at inference/test time
    - Extend it with $\y_\tp$, the *target* (i.e., correct  $\tt^{th}$ word)
    

In other words: to speed up training
- When extending the prefix
- A teacher forces the student (model) to continue with the *correct* answer
- Rather than the student's answer

$$
\x^\ip_\tp = \y_{(t-1)}
$$
for $t > t'$.

<table>
    <tr>
        <th><center>Sequence to Sequence: training (teacher forcing)</center></th>
    </tr>
    <tr>
        <td><img src="images/RNN_seq2seq_training.jpg" width=1000></td>
    </tr>
</table>

Teacher forcing is indicated in red
- Predictions $[ \; \hat{\y}_\tp \, | \, 1 \le \tt \le T \; ]$ **are not** used as input (lower right)
- Only correct targets $[ \; \y_\tp \, | \, 1 \le \tt \le T \; ]$ are used

## Summary

Here is an unrolled graph at inference/test time

<table>
    <tr>
        <th><center>Test time: no forcing</center></th>
    </tr>
    <tr>
        <td><img src="images/RNN_teacher_forcing_inference.jpg" width=800"></td>
    </tr>
</table>

And here is a depiction of the graph used at *training* time

<table>
    <tr>
        <th><center>Training, with Teacher Forcing</center></th>
    </tr>
    <tr>
        <td><img src="images/RNN_teacher_forcing_training.jpg" width=800></td>
    </tr>
</table>

## Generating strange things

Generating stories from seeds was very popular a few years back.

Let's look at some examples.


But first, a surprise:
- Rather than solving a "predict the next word" task
- All of the following examples were generated by a "predict the next **character**" task !


It is somewhat amazing that what is generated
- Has correctly spelled words/keywords
- Is Syntactically correct (sentences end with a ".", parentheses/brackets are balanced)
- Is meaningful: the elements/words are arranged in a logical order

Even though
- We have not explicilty identified any of these concepts
- Nor forced training to respect them (via a loss function)

Remember
- All of this behavior was "learned" by identifying the correct next **character**

- Fake [Shakespeare](http://karpathy.github.io/2015/05/21/rnn-effectiveness/#shakespeare), or fake politician-speak
- Fake code 
- Fake [math textbooks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/#algebraic-geometry-latex)
- [Click bait headline generator](http://clickotron.com/about)


In [2]:
print("Done")

Done
